In [ ]:
# Cell 1: Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import LeaveOneOut, cross_val_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, r2_score
from scipy.optimize import minimize
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

print("Libraries imported successfully!")

In [ ]:
# Cell 2: Load and Explore Data
# Load the Excel file
df = pd.read_excel('FacoDMEK.xlsx', sheet_name='Cleaned Data')

# Display basic information
print(f"Dataset shape: {df.shape}")
print(f"\nColumns: {df.columns.tolist()}")
print(f"\nFirst few rows:")
df.head()

In [ ]:
# Cell 3: Data Preprocessing - Biometry Only
# Calculate average keratometry from BIOMETRY
df['K_avg_Bio'] = (df['Bio-Ks'] + df['Bio-Kf']) / 2

# Calculate the "true" IOL power that would have achieved emmetropia
df['True_IOL'] = df['IOL Power'] - df['PostOP Spherical Equivalent']

# Feature engineering for biometry-based analysis
df['K_Astigmatism_Bio'] = df['Bio-Ks'] - df['Bio-Kf']
df['Post_Ant_Ratio'] = df['Posterior Km'] / df['Anterior Km']
df['CCT_Normalized'] = df['CCT'] / 550  # Normalize CCT

# Display summary statistics
print("Summary of key biometry variables:")
print(df[['Bio-AL', 'K_avg_Bio', 'IOL Power', 'PostOP Spherical Equivalent', 
          'True_IOL', 'CCT', 'Posterior Km']].describe())

# Check for missing values
print("\nMissing values in key columns:")
missing_counts = df[['Bio-AL', 'K_avg_Bio', 'IOL Power', 
                     'PostOP Spherical Equivalent', 'True_IOL', 'CCT', 
                     'A-Constant', 'Posterior Km']].isnull().sum()
print(missing_counts)

# Count complete cases
complete_cases = df[['Bio-AL', 'K_avg_Bio', 'IOL Power', 
                     'PostOP Spherical Equivalent', 'A-Constant']].notna().all(axis=1).sum()
print(f"\nComplete cases for analysis: {complete_cases} out of {len(df)}")

In [ ]:
# Cell 4: Implement SRK/T Formula with Biometry K Values
def calculate_SRKT(AL, K, A_const, nc=1.333):
    """
    Calculate IOL power using SRK/T formula
    Uses biometry K values
    Returns NaN if inputs are invalid
    """
    # Check for valid inputs
    if pd.isna(AL) or pd.isna(K) or pd.isna(A_const) or K <= 0 or AL <= 0:
        return np.nan
    
    try:
        # Constants
        na = 1.336
        V = 12
        
        # Corneal radius from biometry K
        r = 337.5 / K
        
        # Axial length correction - USING NEGATIVE 3.446 AS PER EXCEL FORMULA
        if AL > 24.2:
            LCOR = -3.446 + 1.716 * AL - 0.0237 * AL**2
        else:
            LCOR = AL
        
        # Corneal width
        Cw = -5.41 + 0.58412 * LCOR + 0.098 * K
        
        # Check if we can calculate H (avoid negative square root)
        if r**2 - (Cw**2 / 4) < 0:
            return np.nan
        
        # Corneal height
        H = r - np.sqrt(r**2 - (Cw**2 / 4))
        
        # ACD constant from A-constant
        ACDconst = 0.62467 * A_const - 68.747
        
        # Offset
        offset = ACDconst - 3.336
        
        # Estimated postoperative ACD
        ACDest = H + offset
        
        # Retinal thickness correction
        RETHICK = 0.65696 - 0.02029 * AL
        LOPT = AL + RETHICK
        
        # Calculate IOL power for emmetropia
        ncm1 = nc - 1
        IOL = (1000 * na * (na * r - ncm1 * LOPT)) / ((LOPT - ACDest) * (na * r - ncm1 * ACDest))
        
        return IOL
    except:
        return np.nan

# Calculate SRK/T predictions using biometry K values
df['SRKT_Prediction'] = df.apply(
    lambda row: calculate_SRKT(row['Bio-AL'], row['K_avg_Bio'], row['A-Constant']), 
    axis=1
)
df['SRKT_Error'] = df['SRKT_Prediction'] - df['True_IOL']

# Remove rows with NaN errors for analysis
valid_cases = df['SRKT_Error'].notna()

print("SRK/T with Biometry K Values:")
print(f"Valid predictions: {valid_cases.sum()} out of {len(df)}")
if valid_cases.sum() > 0:
    print(f"Mean Error: {df.loc[valid_cases, 'SRKT_Error'].mean():.3f} D")
    print(f"Mean Absolute Error: {df.loc[valid_cases, 'SRKT_Error'].abs().mean():.3f} D")
    print(f"Standard Deviation: {df.loc[valid_cases, 'SRKT_Error'].std():.3f} D")

In [ ]:
# Cell 5: Visualize SRK/T Performance with Biometry
# Only plot for rows with valid data
df_valid = df[df['SRKT_Error'].notna()].copy()

fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Plot 1: Error Distribution
if len(df_valid) > 0:
    axes[0, 0].hist(df_valid['SRKT_Error'], bins=20, edgecolor='black', alpha=0.7)
    axes[0, 0].axvline(x=0, color='red', linestyle='--')
    axes[0, 0].set_xlabel('SRK/T Prediction Error (D)')
    axes[0, 0].set_ylabel('Frequency')
    axes[0, 0].set_title(f'Error Distribution (n={len(df_valid)})')

# Plot 2: Error vs Axial Length
    axes[0, 1].scatter(df_valid['Bio-AL'], df_valid['SRKT_Error'], alpha=0.6)
    axes[0, 1].set_xlabel('Axial Length (mm)')
    axes[0, 1].set_ylabel('SRK/T Error (D)')
    axes[0, 1].set_title('Error vs Axial Length')
    axes[0, 1].axhline(y=0, color='red', linestyle='--', alpha=0.5)

# Plot 3: Error vs Posterior Corneal Power
    axes[1, 0].scatter(df_valid['Posterior Km'], df_valid['SRKT_Error'], alpha=0.6)
    axes[1, 0].set_xlabel('Posterior Corneal Power (D)')
    axes[1, 0].set_ylabel('SRK/T Error (D)')
    axes[1, 0].set_title('Error vs Posterior K')
    axes[1, 0].axhline(y=0, color='red', linestyle='--', alpha=0.5)

# Plot 4: Error vs CCT
    valid_cct = df_valid[df_valid['CCT'].notna()]
    axes[1, 1].scatter(valid_cct['CCT'], valid_cct['SRKT_Error'], alpha=0.6)
    axes[1, 1].set_xlabel('Central Corneal Thickness (μm)')
    axes[1, 1].set_ylabel('SRK/T Error (D)')
    axes[1, 1].set_title('Error vs CCT')
    axes[1, 1].axhline(y=0, color='red', linestyle='--', alpha=0.5)

plt.tight_layout()
plt.show()

# Calculate percentage within target ranges
if len(df_valid) > 0:
    within_025 = (df_valid['SRKT_Error'].abs() <= 0.25).sum() / len(df_valid) * 100
    within_050 = (df_valid['SRKT_Error'].abs() <= 0.50).sum() / len(df_valid) * 100
    within_100 = (df_valid['SRKT_Error'].abs() <= 1.00).sum() / len(df_valid) * 100
    
    print(f"\nPercentage of eyes within target:")
    print(f"±0.25 D: {within_025:.1f}%")
    print(f"±0.50 D: {within_050:.1f}%")
    print(f"±1.00 D: {within_100:.1f}%")

In [ ]:
# Cell 6: Optimization Approach 1 - Optimize Corneal Refractive Index
# Only use complete cases
df_complete = df[df['SRKT_Error'].notna()].copy()

def objective_nc(nc_value):
    """Objective function to minimize MAE by optimizing nc"""
    predictions = df_complete.apply(
        lambda row: calculate_SRKT(row['Bio-AL'], row['K_avg_Bio'], row['A-Constant'], nc=nc_value[0]), 
        axis=1
    )
    errors = predictions - df_complete['True_IOL']
    # Remove any NaN values that might occur
    valid_errors = errors[errors.notna()]
    if len(valid_errors) == 0:
        return 999  # Return large value if no valid predictions
    return np.mean(np.abs(valid_errors))

# Optimize nc
if len(df_complete) > 0:
    result_nc = minimize(objective_nc, x0=[1.333], bounds=[(1.330, 1.340)], method='L-BFGS-B')
    optimal_nc = result_nc.x[0]
    
    # Recalculate with optimal nc
    df_complete['SRKT_Optimized_nc'] = df_complete.apply(
        lambda row: calculate_SRKT(row['Bio-AL'], row['K_avg_Bio'], row['A-Constant'], nc=optimal_nc), 
        axis=1
    )
    df_complete['SRKT_Error_Optimized_nc'] = df_complete['SRKT_Optimized_nc'] - df_complete['True_IOL']
    
    print(f"Optimal corneal refractive index: {optimal_nc:.4f}")
    print(f"Original MAE: {df_complete['SRKT_Error'].abs().mean():.3f} D")
    print(f"Optimized MAE: {df_complete['SRKT_Error_Optimized_nc'].abs().mean():.3f} D")
else:
    print("Not enough complete cases for optimization")

In [ ]:
# Cell 7: Linear Correction Model Based on Biometry and OCT Parameters
# Prepare features for correction model
feature_cols = ['Posterior Km', 'CCT', 'Post_Ant_Ratio', 'K_Astigmatism_Bio']

# Create a dataset with only complete cases
df_ml = df.copy()

# Ensure we have all necessary columns
required_cols = feature_cols + ['SRKT_Error']
df_ml = df_ml[df_ml[required_cols].notna().all(axis=1)].copy()

print(f"Complete cases for ML: {len(df_ml)} out of {len(df)}")

if len(df_ml) > 10:  # Need at least 10 cases for meaningful analysis
    X = df_ml[feature_cols]
    y = df_ml['SRKT_Error']
    
    # Split data for cross-validation
    loo = LeaveOneOut()
    predictions_linear = []
    true_values = []
    
    for train_index, test_index in loo.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        # Train linear model
        model = LinearRegression()
        model.fit(X_train, y_train)
        
        # Predict correction
        correction = model.predict(X_test)[0]
        
        # Apply correction to SRK/T prediction
        srkt_pred = df_ml.iloc[test_index]['SRKT_Prediction'].values[0]
        corrected_pred = srkt_pred - correction
        
        predictions_linear.append(corrected_pred)
        true_values.append(df_ml.iloc[test_index]['True_IOL'].values[0])
    
    # Calculate performance
    mae_original = df_ml['SRKT_Error'].abs().mean()
    mae_linear = mean_absolute_error(true_values, predictions_linear)
    
    print(f"Original SRK/T MAE: {mae_original:.3f} D")
    print(f"Linear Correction Model MAE: {mae_linear:.3f} D")
    print(f"Improvement: {mae_original - mae_linear:.3f} D")
    
    # Show feature importance
    model_full = LinearRegression()
    model_full.fit(X, y)
    feature_importance = pd.DataFrame({
        'Feature': feature_cols,
        'Coefficient': model_full.coef_,
        'Abs_Coefficient': np.abs(model_full.coef_)
    }).sort_values('Abs_Coefficient', ascending=False)
    
    print("\nFeature Importance (Linear Model):")
    print(feature_importance)
else:
    print("Not enough complete cases for machine learning analysis")

In [ ]:
# Cell 8: Machine Learning Models Comparison
if len(df_ml) > 10:
    # Prepare all biometry-based features
    features_ml = ['Bio-AL', 'K_avg_Bio', 'Posterior Km', 'CCT', 'Post_Ant_Ratio', 
                   'K_Astigmatism_Bio', 'A-Constant']
    
    # Ensure complete cases
    df_ml_full = df_ml[features_ml + ['True_IOL']].dropna()
    X_ml = df_ml_full[features_ml]
    y_ml = df_ml_full['True_IOL']
    
    print(f"Cases for ML comparison: {len(X_ml)}")
    
    # Initialize models
    models = {
        'Linear Regression': LinearRegression(),
        'Ridge Regression': Ridge(alpha=1.0),
        'Random Forest': RandomForestRegressor(n_estimators=100, max_depth=5, random_state=42),
        'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, max_depth=3, random_state=42)
    }
    
    # Cross-validation results
    results = {}
    
    for name, model in models.items():
        predictions = []
        actuals = []
        
        for train_idx, test_idx in loo.split(X_ml):
            X_train, X_test = X_ml.iloc[train_idx], X_ml.iloc[test_idx]
            y_train, y_test = y_ml.iloc[train_idx], y_ml.iloc[test_idx]
            
            model.fit(X_train, y_train)
            pred = model.predict(X_test)[0]
            
            predictions.append(pred)
            actuals.append(y_test.values[0])
        
        mae = mean_absolute_error(actuals, predictions)
        results[name] = mae
        print(f"{name} MAE: {mae:.3f} D")
    
    # Compare with original SRK/T
    original_mae = df_ml_full.merge(df[['SRKT_Error']], left_index=True, right_index=True)['SRKT_Error'].abs().mean()
    print(f"\nOriginal SRK/T MAE: {original_mae:.3f} D")
    print(f"Best ML Model: {min(results, key=results.get)} with MAE: {min(results.values()):.3f} D")

In [ ]:
# Cell 9: Create Final Optimized Formula for FacoDMEK
if len(df_ml) > 10:
    # Train final model on all data
    best_model = GradientBoostingRegressor(n_estimators=100, max_depth=3, random_state=42)
    best_model.fit(X_ml, y_ml)
    
    # Feature importance
    feature_importance_gb = pd.DataFrame({
        'Feature': features_ml,
        'Importance': best_model.feature_importances_
    }).sort_values('Importance', ascending=False)
    
    plt.figure(figsize=(10, 6))
    plt.barh(feature_importance_gb['Feature'], feature_importance_gb['Importance'])
    plt.xlabel('Feature Importance')
    plt.title('Feature Importance in IOL Power Prediction Model')
    plt.tight_layout()
    plt.show()
    
    print("Feature Importance:")
    print(feature_importance_gb)
    
    # Create simplified correction formula based on key biometry/OCT features
    top_features = ['Posterior Km', 'CCT', 'Bio-AL']
    
    # Ensure we have complete data for these features
    df_simple = df_ml[top_features + ['True_IOL', 'SRKT_Prediction']].dropna()
    X_simple = df_simple[top_features]
    y_correction = df_simple['True_IOL'] - df_simple['SRKT_Prediction']
    
    model_simple = LinearRegression()
    model_simple.fit(X_simple, y_correction)
    
    print(f"\nFacoDMEK Correction Formula (Biometry-Based):")
    print(f"Correction = {model_simple.intercept_:.3f}")
    for feat, coef in zip(top_features, model_simple.coef_):
        print(f"           + {coef:.4f} × {feat}")
    
    # Calculate final performance
    df_simple['Predicted_Correction'] = model_simple.predict(X_simple)
    df_simple['Final_Prediction'] = df_simple['SRKT_Prediction'] + df_simple['Predicted_Correction']
    final_mae = mean_absolute_error(df_simple['True_IOL'], df_simple['Final_Prediction'])
    
    print(f"\nFinal formula MAE: {final_mae:.3f} D")

In [ ]:
# Cell 10: Final Validation and Results Summary
if len(df_ml) > 10:
    # Summary statistics
    print("FINAL RESULTS SUMMARY - BIOMETRY-BASED ANALYSIS")
    print("="*60)
    print(f"Original data points: {len(df)}")
    print(f"Complete cases for analysis: {len(df_ml)}")
    print(f"\nOriginal SRK/T Performance (Biometry K):")
    print(f"  MAE: {df_ml['SRKT_Error'].abs().mean():.3f} D")
    print(f"  Mean Error: {df_ml['SRKT_Error'].mean():.3f} D")
    print(f"  STD: {df_ml['SRKT_Error'].std():.3f} D")
    
    if 'optimal_nc' in locals():
        print(f"\nOptimized corneal refractive index: {optimal_nc:.4f}")
    
    print("\n" + "="*60)
    print("RECOMMENDED FACODMEK FORMULA (BIOMETRY-BASED):")
    print("="*60)
    print("Modified_IOL = SRK/T_Biometry + Correction")
    print(f"\nWhere Correction = {model_simple.intercept_:.3f} + "
          f"{model_simple.coef_[0]:.4f}×Posterior_Km + "
          f"{model_simple.coef_[1]:.4f}×CCT + "
          f"{model_simple.coef_[2]:.4f}×AL")
    
    print("\nNote: Uses biometry K values (not topography)")
    print("      Posterior K from OCT is critical for accuracy")
    
    # Save results
    results_df = df_ml[['ID', 'Patient', 'Eye', 'Bio-AL', 'K_avg_Bio', 'IOL Power', 
                        'PostOP Spherical Equivalent', 'True_IOL', 'SRKT_Prediction', 
                        'SRKT_Error', 'Posterior Km', 'CCT']].copy()
    
    results_df.to_excel('FacoDMEK_Biometry_Optimization_Results.xlsx', index=False)
    print("\nResults saved to 'FacoDMEK_Biometry_Optimization_Results.xlsx'")
    
    # Create visualization of improvement
    plt.figure(figsize=(10, 6))
    
    # Bland-Altman plot
    mean_values = (df_simple['True_IOL'] + df_simple['Final_Prediction']) / 2
    diff_values = df_simple['Final_Prediction'] - df_simple['True_IOL']
    
    plt.scatter(mean_values, diff_values, alpha=0.6)
    plt.axhline(y=0, color='red', linestyle='-', linewidth=2)
    plt.axhline(y=diff_values.mean(), color='blue', linestyle='--')
    plt.axhline(y=diff_values.mean() + 1.96*diff_values.std(), color='blue', linestyle=':')
    plt.axhline(y=diff_values.mean() - 1.96*diff_values.std(), color='blue', linestyle=':')
    
    plt.xlabel('Mean of True and Predicted IOL (D)')
    plt.ylabel('Predicted - True IOL (D)')
    plt.title('Bland-Altman Plot: FacoDMEK Formula Performance')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
else:
    print("Insufficient data for complete analysis")

In [ ]:
# Cell 11: Extract Formula from Ridge Regression Model
if len(df_ml) > 10:
    # First, let's train Ridge on predicting the CORRECTION needed, not the absolute IOL
    # This makes the formula more interpretable and clinically useful
    
    # Prepare data
    df_ridge = df_ml[features_ml + ['True_IOL', 'SRKT_Prediction']].dropna()
    X_ridge = df_ridge[features_ml]
    
    # Calculate the correction needed (True IOL - SRK/T prediction)
    y_correction = df_ridge['True_IOL'] - df_ridge['SRKT_Prediction']
    
    # Train Ridge Regression for correction
    ridge_model = Ridge(alpha=1.0)
    ridge_model.fit(X_ridge, y_correction)
    
    # Extract coefficients
    print("RIDGE REGRESSION CORRECTION FORMULA:")
    print("="*60)
    print(f"Correction = {ridge_model.intercept_:.4f}")
    
    # Create a coefficient table
    coef_df = pd.DataFrame({
        'Feature': features_ml,
        'Coefficient': ridge_model.coef_,
        'Abs_Coefficient': np.abs(ridge_model.coef_)
    }).sort_values('Abs_Coefficient', ascending=False)
    
    print("\nCoefficients (sorted by importance):")
    print(coef_df)
    
    # Create the practical formula
    print("\n" + "="*60)
    print("PRACTICAL FACODMEK FORMULA FROM RIDGE REGRESSION:")
    print("="*60)
    print("Modified_IOL = Standard_SRK/T + Correction")
    print(f"\nWhere Correction = {ridge_model.intercept_:.3f}")
    
    for feat, coef in zip(features_ml, ridge_model.coef_):
        if abs(coef) > 0.001:  # Only show meaningful coefficients
            print(f"    + ({coef:.4f} × {feat})")
    
    # Simplified version using only the most important features
    print("\n" + "="*60)
    print("SIMPLIFIED FORMULA (Top 3 Features):")
    print("="*60)
    
    # Get top 3 features by absolute coefficient value
    top_3_indices = coef_df.nlargest(3, 'Abs_Coefficient').index
    top_3_features = [features_ml[i] for i in top_3_indices]
    top_3_coefs = [ridge_model.coef_[i] for i in top_3_indices]
    
    # Re-fit with only top 3 features for a cleaner formula
    X_simple_ridge = df_ridge[top_3_features]
    ridge_simple = Ridge(alpha=1.0)
    ridge_simple.fit(X_simple_ridge, y_correction)
    
    print("Modified_IOL = Standard_SRK/T + Correction")
    print(f"\nWhere Correction = {ridge_simple.intercept_:.3f}")
    for feat, coef in zip(top_3_features, ridge_simple.coef_):
        print(f"    + ({coef:.4f} × {feat})")
    
    # Validate the simplified formula
    y_pred_simple = ridge_simple.predict(X_simple_ridge)
    final_iol_simple = df_ridge['SRKT_Prediction'] + y_pred_simple
    mae_simple = mean_absolute_error(df_ridge['True_IOL'], final_iol_simple)
    
    print(f"\nSimplified formula MAE: {mae_simple:.3f} D")
    
    # Create a clinically usable formula card
    print("\n" + "="*60)
    print("CLINICAL FORMULA CARD - FACODMEK IOL CALCULATION")
    print("="*60)
    print("Step 1: Calculate standard SRK/T using biometry K")
    print("Step 2: Apply correction:")
    print(f"    Correction = {ridge_simple.intercept_:.2f}")
    for feat, coef in zip(top_3_features, ridge_simple.coef_):
        if feat == 'Bio-AL':
            print(f"    + ({coef:.3f} × AL in mm)")
        elif feat == 'K_avg_Bio':
            print(f"    + ({coef:.3f} × Mean K from biometry in D)")
        elif feat == 'Posterior Km':
            print(f"    + ({coef:.3f} × Posterior K from OCT in D)")
        elif feat == 'CCT':
            print(f"    + ({coef:.4f} × CCT in μm)")
        elif feat == 'A-Constant':
            print(f"    + ({coef:.3f} × A-constant)")
    print("Step 3: Modified IOL = SRK/T + Correction")